# Import software libraries and load the dataset

In [ ]:
import sys                             # Read system parameters.
import shutil
import numpy as np                     # Work with multi-dimensional arrays and matrices.
from numpy.random import seed
import matplotlib as mpl               # Create 2D charts.
import matplotlib.pyplot as plt
import sklearn                         # Perform data mining and analysis.
import tensorflow                      # Train neural networks for deep learning.
import keras                           # Provide a frontend for TensorFlow.
from keras import datasets

# Summarize software libraries used.
print('Libraries used in this project:')
print('- Python {}'.format(sys.version))
print('- NumPy {}'.format(np.__version__))
print('- Matplotlib {}'.format(mpl.__version__))
print('- scikit-learn {}'.format(sklearn.__version__))
print('- TensorFlow {}'.format(tensorflow.__version__))
print('- Keras {}\n'.format(keras.__version__))

# Load the dataset.
shutil.rmtree('/home/jovyan/.keras')
shutil.copytree('/home/jovyan/work/.keras', '/home/jovyan/.keras')
(X_train, y_train), (X_test, y_test) = datasets.imdb.load_data(num_words = 10000)
print('Loaded {} training records.'.format(len(X_train.data)))
print('Loaded {} test records.'.format(len(X_test.data)))

# Uncomment the following two lines to make outcomes deterministic. Supply whatever seed values you wish.
#seed(1)
#tensorflow.random.set_seed(1)

# Get acquainted with the dataset

In [ ]:
print('First example features:\n')
print(X_train[0])
print('\n')
print('Label: {}'.format(y_train[0]))

In [ ]:
# Decode sequence values into actual text.
index = datasets.imdb.get_word_index()
index_dict = dict([(value, key) for (key, value) in index.items()])
decode = ' '.join([index_dict.get(i - 3, '?') for i in X_train[0]])  # Replace unknown words with '?'.
print(decode) 

# Examine some statistics about the reviews

In [ ]:
result = [len(i) for i in X_train]
print('Mean review length (in words): {:.0f}'.format(np.mean(result)))
print('Standard deviation (in words): {:.0f}'.format(np.std(result)))

In [ ]:
plt.figure(figsize = (15, 2))
plt.boxplot(result, vert = False)
plt.xlabel('Review length (in words)')
plt.show()

# Add padding to the data

In [ ]:
from keras.preprocessing import sequence

X_train = sequence.pad_sequences(X_train, maxlen = 500)
X_test = sequence.pad_sequences(X_test, maxlen = 500)

print('Number of features: {}'.format(X_train.shape[1]))

# Split the datasets

In [ ]:
from sklearn.model_selection import train_test_split

# Split the training and validation datasets and their labels.
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state = 50)

print(f'Training features:         {X_train.shape}')
print(f'Validation features:       {X_val.shape}')
print(f'Training labels:           {y_train.shape}')
print(f'Validation labels:         {y_val.shape}')

# Build the RNN structure

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from keras.layers.advanced_activations import LeakyReLU

rnn = Sequential()

# Start stacking layers one-by-one.
rnn.add(Embedding(input_dim = 10000,  # Size of vocabulary (top 10,000 words).
                  output_dim = 100,  # 100-dimensional vector embedding.
                  input_length = 500))  # Length of review (in words).

rnn.add(LSTM(units = 64))  # 64-dimensional LSTM.
rnn.add(LeakyReLU(alpha = 0.1))

rnn.add(Dense(128, activation = 'linear'))
rnn.add(LeakyReLU(alpha = 0.1))
rnn.add(Dense(1, activation = 'sigmoid'))  # Dense output layer with sigmoid activation.

print('The RNN structure has been built.')

# Compile the model and examine the layers

In [ ]:
rnn.compile(optimizer = 'adam',
            loss = 'binary_crossentropy',
            metrics = ['accuracy'])

rnn.summary()

In [ ]:
# Install the required library.
!conda install --yes graphviz==2.40.1

In [ ]:
from keras.utils import plot_model
plot_model(rnn, show_shapes = True, to_file = 'model2.png')

# Train the model

In [ ]:
rnn_trained = rnn.fit(X_train, y_train,
                      validation_data = (X_val, y_val),
                      epochs = 1,
                      verbose = 1)

# Evaluate the model on the test data

In [ ]:
eval_test = rnn.evaluate(X_test, y_test, verbose = 0)

print('Loss: {}'.format(round(eval_test[0], 2)))
print('Accuracy: {:.0f}%'.format(eval_test[1] * 100))

# Make predictions on the test data

In [ ]:
prediction = rnn.predict(X_test[:100])  # Predict first 100 examples to save time.
prediction = np.round(np.ndarray.flatten(prediction))  # Round probabilities to 0 or 1.
prediction = prediction.astype(int)

print('Actual class:    {}'.format(y_test[:10]))
print('Predicted class: {}'.format(prediction[:10]))

# Examine a review that was correctly classified

In [ ]:
label_names = ['negative', 'positive']

for i in range(100):
    if y_test[i] == prediction[i]:
        print('Actual review sentiment:    {}'.format(label_names[y_test[i]]))
        print('Predicted review sentiment: {}\n'.format(label_names[prediction[i]]))
        
        decode = ' '.join([index_dict.get(x - 3, '?') for x in X_test[i]])
        
        print(decode)
        
        break

# Examine a review that was incorrectly classified

In [ ]:
for i in range(100):
    if y_test[i] != prediction[i]:
        print('Actual review sentiment:    {}'.format(label_names[y_test[i]]))
        print('Predicted review sentiment: {}\n'.format(label_names[prediction[i]]))
        
        decode = ' '.join([index_dict.get(x - 3, '?') for x in X_test[i]])
        
        print(decode)
        
        break